In [3]:
#importing libraries
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
import numpy as np
import os

In [4]:
block_size = 10

In [5]:
# Dataset
with open('/Users/nimitt/Documents/DigitalSystems/Project/Datasets/advisories.txt','r') as file:
    text = file.read()
text = text.lower()

chars = ['\x00',
 '\t',
 '\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~',
 '\xa0',
 '§',
 '·',
 'ं',
 'क',
 'ग',
 'त',
 'थ',
 'द',
 'ध',
 'न',
 'प',
 'भ',
 'य',
 'र',
 'स',
 'ा',
 'ि',
 'ी',
 'ो',
 'ौ',
 '्',
 '–',
 '‘',
 '’',
 '“',
 '”',
 '„',
 '‟',
 '•',
 '…',
 '\u2028',
 '₹',
 '→',
 '∑',
 '√',
 '≥',
 '□',
 '▪',
 '○',
 '●',
 '➢',
 '\uf020',
 '\uf02d',
 '\uf0a7',
 '\uf0b7',
 '\uf0d8',
 '\uf0fc',
 'ﬀ',
 'ﬁ',
 'ﬃ',
 '𝐶',
 '𝐼',
 '𝑃',
 '𝑆',
 '𝑐',
 '𝑔']


#  Removing unwanted chars
unwanted_chars = chars[68:] + chars[:20]
for unwanted_char in unwanted_chars:
    if (unwanted_char in ['•',
                            '…',
                            '\u2028',
                            '₹',
                            '→',
                            '∑','√','≥','□','▪','○','●','➢','\uf020','\uf02d','\uf0a7','\uf0b7','\uf0d8','\uf0fc',]):
        text = text.replace(unwanted_char,"|")
    elif (unwanted_char in [ '‘',
 '’',
 '“',
 '”',
 '„',
 '‟',]):
        text = text.replace(unwanted_char,"'")
    else:
        text = text.replace(unwanted_char,"~")
text = text.replace('\n',"~")
text = text.replace('\t',"~")
text = text.replace('\x00',"~")
text_len = 500
text = text[:text_len]
# Vocabulary
chars = sorted(set(text))   

stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

# Creating X and Y
X, Y = [],[]
context = []
for j in range(block_size):
    context = context + [stoi[text[j]]]

for i in range(block_size, len(text)-1):

    X_ = np.zeros((block_size,len(chars)))

    for j in range(block_size):
        X_[j][context[j]] = 1

    ch = text[i]
    ix = stoi[ch]


    X.append(X_)
    
    context = context[1:] + [ix] 
    Y.append(context)

X = torch.tensor(np.array(X),dtype=torch.float32)
Y = torch.tensor(np.array(Y),dtype = torch.long)

In [10]:
stoi

{'1': 0,
 '2': 1,
 '3': 2,
 '5': 3,
 'a': 4,
 'b': 5,
 'c': 6,
 'd': 7,
 'e': 8,
 'f': 9,
 'g': 10,
 'h': 11,
 'i': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'y': 25,
 '~': 26}

In [6]:
itos

{0: '1',
 1: '2',
 2: '3',
 3: '5',
 4: 'a',
 5: 'b',
 6: 'c',
 7: 'd',
 8: 'e',
 9: 'f',
 10: 'g',
 11: 'h',
 12: 'i',
 13: 'k',
 14: 'l',
 15: 'm',
 16: 'n',
 17: 'o',
 18: 'p',
 19: 'r',
 20: 's',
 21: 't',
 22: 'u',
 23: 'v',
 24: 'w',
 25: 'y',
 26: '~'}

In [11]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Forget Gate
        self.wf = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.bf = nn.Parameter(torch.Tensor(hidden_size, 1))

        # Input Gate
        self.wi = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.bi = nn.Parameter(torch.Tensor(hidden_size, 1))

        # Candidate Gate
        self.wc = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.bc = nn.Parameter(torch.Tensor(hidden_size, 1))

        # Output Gate
        self.wo = nn.Parameter(torch.Tensor(hidden_size, input_size))
        self.bo = nn.Parameter(torch.Tensor(hidden_size, 1))

        # Final Gate
        self.wy = nn.Parameter(torch.Tensor(output_size, hidden_size))
        self.by = nn.Parameter(torch.Tensor(output_size, 1))

        # Initialize parameters
        self.init_parameters()

    def init_parameters(self):
        # Initialize weights with Xavier initialization
        nn.init.xavier_uniform_(self.wf)
        nn.init.xavier_uniform_(self.wi)
        nn.init.xavier_uniform_(self.wc)
        nn.init.xavier_uniform_(self.wo)
        nn.init.xavier_uniform_(self.wy)

        # Initialize biases to zeros
        nn.init.constant_(self.bf, 0)
        nn.init.constant_(self.bi, 0)
        nn.init.constant_(self.bc, 0)
        nn.init.constant_(self.bo, 0)
        nn.init.constant_(self.by, 0)
        

    def forward(self, X):
        outputs = []
        seq_length = X.size(0)
        hidden_state = torch.zeros(self.hidden_size,1,dtype=torch.float32)
        cell_state = torch.zeros(self.hidden_size,1,dtype = torch.float32)

        for q in range(seq_length):
            concat_input = torch.cat((hidden_state, X[q].unsqueeze(1)), dim=0)
            forget_gate = torch.sigmoid(torch.matmul(self.wf, concat_input) + self.bf)
            input_gate = torch.sigmoid(torch.matmul(self.wi, concat_input) + self.bi)
            candidate_gate = torch.tanh(torch.matmul(self.wc, concat_input) + self.bc)
            output_gate = torch.sigmoid(torch.matmul(self.wo, concat_input) + self.bo)

            cell_state = forget_gate * cell_state + input_gate * candidate_gate
            hidden_state = output_gate * torch.tanh(cell_state)

            output = torch.matmul(self.wy, hidden_state) + self.by
            outputs.append(output)
        outputs = torch.stack(outputs)
        return outputs
    
    def predict(self, X):
        out = [self.forward(x) for x in X]
        return torch.stack(out)
    
    def train(self, X, y, epochs, lr, model_path):
        optimizer = optim.Adam(self.parameters(), lr=lr)
        criterion = torch.nn.CrossEntropyLoss()  

        for epoch in range(epochs):
            epoch_loss = 0.0
            optimizer.zero_grad() 

            prediction = self.predict(X)
            prediction = prediction.reshape(-1, self.output_size)
            target = y.reshape(-1)
            loss = criterion(prediction, target)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()


            current_loss = epoch_loss / len(X)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {current_loss:.4f}")

            if (epoch % 100 == 0):
                torch.save({
                            'epoch': epoch,
                            'model_state_dict': self.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'loss': loss,
                            }, model_path)

In [12]:
input_size = 27
hidden_size = 25

In [13]:
# input_size = len(chars)

model = LSTM(hidden_size+input_size, hidden_size, input_size)

In [14]:
# Training

# checkpoint_folder = "/Users/nimitt/Documents/DigitalSystems/Project/Model_States"
# model_filename = f"model_{block_size}_{hidden_size}_{text_len}.pt"
# model_path = os.path.join(checkpoint_folder, model_filename)
# # Ensure the folder exists
# os.makedirs(checkpoint_folder, exist_ok=True)

# model.train(X,Y,500,0.01,model_path)

Epoch 1/500, Loss: 0.0067
Epoch 2/500, Loss: 0.0067
Epoch 3/500, Loss: 0.0066
Epoch 4/500, Loss: 0.0065
Epoch 5/500, Loss: 0.0064
Epoch 6/500, Loss: 0.0063
Epoch 7/500, Loss: 0.0062
Epoch 8/500, Loss: 0.0060
Epoch 9/500, Loss: 0.0059
Epoch 10/500, Loss: 0.0058
Epoch 11/500, Loss: 0.0058
Epoch 12/500, Loss: 0.0057
Epoch 13/500, Loss: 0.0057
Epoch 14/500, Loss: 0.0056
Epoch 15/500, Loss: 0.0055
Epoch 16/500, Loss: 0.0055
Epoch 17/500, Loss: 0.0055
Epoch 18/500, Loss: 0.0054
Epoch 19/500, Loss: 0.0054
Epoch 20/500, Loss: 0.0054
Epoch 21/500, Loss: 0.0053
Epoch 22/500, Loss: 0.0053
Epoch 23/500, Loss: 0.0053
Epoch 24/500, Loss: 0.0052
Epoch 25/500, Loss: 0.0052
Epoch 26/500, Loss: 0.0051
Epoch 27/500, Loss: 0.0051
Epoch 28/500, Loss: 0.0051
Epoch 29/500, Loss: 0.0050
Epoch 30/500, Loss: 0.0050
Epoch 31/500, Loss: 0.0049
Epoch 32/500, Loss: 0.0049
Epoch 33/500, Loss: 0.0049
Epoch 34/500, Loss: 0.0048
Epoch 35/500, Loss: 0.0048
Epoch 36/500, Loss: 0.0047
Epoch 37/500, Loss: 0.0047
Epoch 38/5

In [11]:
# # Testing

# def convert_to_X(prompt):
#     X_ = np.zeros((len(prompt),input_size))
#     for i in range(len(prompt)):
#         X_[i][stoi[prompt[i]]] = 1
#     return torch.tensor(X_,dtype = torch.float32)
        
# prompt = "alumni"
# max_len = 100
# context = []
# for j in range(len(prompt)):
#     context = context + [stoi[prompt[j]]]
# context = context[-block_size:]

# generated_text = prompt
# for i in range(max_len):
#     x = convert_to_X(generated_text)
#     y_pred = model(x)[-1]
#     # ix = torch.distributions.categorical.Categorical(logits=y_pred.squeeze()).sample()
#     ix = torch.argmax(y_pred)
#     ch = itos[ix.item()]
#     generated_text += ch
#     context = context[1:] + [ix]

# genrated_text = generated_text.replace('|','\n')
# print(generated_text)

alumni                                                                                                    


In [ ]:
text[:500]

'alumni relations advisory 02 (version 1.0, december 2021)                                 page 1 of 1 ~ indian institute of technology gandhinagar  ~ ~honorary  alumni program at iit gandhinagar  ~(as approved by the bog in its 33rd meeting held on 15 november 2021)  ~ ~ ~the board of governors of the institute in its 33rd meeting held on 15 november ~2021 approved the following honorary alumni program  at iit gandhinagar.  ~  ~1. individuals who are not graduates of iit gandhinagar and who make'

In [16]:
model_read = LSTM(input_size+hidden_size, hidden_size, input_size)
model_path = "/Users/nimitt/Documents/DigitalSystems/Project/NextWordPredictor-Python/SmallModelStates/model_10_25_500.pt"
checkpoint = torch.load(model_path,map_location=torch.device('cpu'))
model_read.load_state_dict(checkpoint['model_state_dict'])

# Testing

def convert_to_X(prompt):
    X_ = np.zeros((len(prompt),input_size))
    for i in range(len(prompt)):
        X_[i][stoi[prompt[i]]] = 1
    return torch.tensor(X_,dtype = torch.float32)
        
prompt = "basi"
max_len = 50
context = []
for j in range(len(prompt)):
    context = context + [stoi[prompt[j]]]
context = context[-block_size:]

generated_text = prompt
for i in range(max_len):
    x = convert_to_X(generated_text)
    y_pred = model_read(x)[-1]
    # ix = torch.distributions.categorical.Categorical(logits=y_pred.squeeze()).sample()
    ix = torch.argmax(y_pred)
    ch = itos[ix.item()]
    generated_text += ch
    context = context[1:] + [ix]

genrated_text = generated_text.replace('|','\n')
print("|",generated_text,"|")

| basiapproved~the~following~honorary~~alumni~program~at |


In [17]:
################################# Writing the Parameters ##################
parameter_list = [model_read.wf, 
model_read.bf,
model_read.wi, 
model_read.bi,
model_read.wc, 
model_read.bc,
model_read.wo, 
model_read.bo, 
model_read.wy, 
model_read.by]

parameter_names = ['wf','bf','wi','bi','wc','bc','wo','bo','wy','by']

In [18]:
import numpy as np

parameter_list = [model_read.wf, model_read.bf, model_read.wi, model_read.bi, model_read.wc,
                  model_read.bc, model_read.wo, model_read.bo, model_read.wy, model_read.by]

parameter_names = ['wf', 'bf', 'wi', 'bi', 'wc', 'bc', 'wo', 'bo', 'wy', 'by']

with open("/Users/nimitt/Documents/DigitalSystems/Project/NextWordPredictor-Python/SmallModelStates/parameters_10_25.txt", 'w') as file:
    file.write("Model Parameters\n")
    file.write("================\n\n")
    for name, parameter in zip(parameter_names, parameter_list):
        file.write(name + ":\n")
        # Convert tensor to NumPy array
        parameter_array = parameter.detach().numpy()
        # Write each value of the parameter tensor to the file
        for row in parameter_array:
            row_str = " ".join([str(val) for val in row])
            file.write(row_str + "\n")
        file.write("\n")


In [19]:
import struct

In [22]:
with open("/Users/nimitt/Documents/DigitalSystems/Project/NextWordPredictor-Python/SmallModelStates/parameters_10_25.txt",'r') as file: 
    params_data = file.read()

In [23]:
params_data = params_data.split('\n')

In [24]:
parameters = {'wf':[],'bf':[],'wi':[],'bi':[],'wc':[],'bc':[],'wo':[],'bo':[],'wy':[],'by':[]}
current_parameter = ''
for i in range(3, len(params_data)):
    if (params_data[i] == ''):
        continue
    if (params_data[i][:2] in parameters.keys()):
        current_parameter = params_data[i][:2]
        continue
    parameters[current_parameter].append(list(map(float,params_data[i].split())))

In [25]:
wf = parameters['wf']
bf = parameters['bf']
wi = parameters['wi']
bi= parameters['bi']
wc = parameters['wc']
bc= parameters['bc']
wo = parameters['wo']
bo = parameters['bo']
wy = parameters['wy']
by= parameters['by']

In [26]:
################ converting parameters to 32bit IEEE 754 #####################
def float_to_binary_IEEE754(f):
    packed = struct.pack('>f', f)
    binary = ''.join(format(byte, '08b') for byte in packed)
    
    return binary

In [27]:
parameters = [wf,bf,wi,bi,wo,bo,wc,bc,wy,by]
parameters_flatted = []
for parameter in parameters:
    for i in range(len(parameter)):
        for j in range(len(parameter[0])):
            parameters_flatted.append(float_to_binary_IEEE754(parameter[i][j]))

In [28]:
len(parameters_flatted)

6002

In [32]:
with open("/Users/nimitt/Documents/DigitalSystems/Project/NextWordPredictor-Python/SmallModelStates/parameters_10_25_ieee.txt",'w') as file:
    for i in range(len(parameters_flatted)):
        file.write(parameters_flatted[i])
        file.write(",")
        file.write('\n')

In [33]:
for parameter in parameters:
    print(len(parameter),len(parameter[0]))

25 52
25 1
25 52
25 1
25 52
25 1
25 52
25 1
27 25
27 1
